# Frontend Bug Fixes: To-Do App with Poem Generation

This notebook explains the problems we encountered in our React/Next.js To-Do application and how we fixed them. These are common issues that beginner developers face when working with CSS and React components.

## Overview of the Application

Our To-Do app has a feature where users can click a "Generate Poem" button to create a poem about their todo item. The poem should appear in a **popup overlay** (also called a modal) in the center of the screen.

**The component structure:**
```
components/
├── todo.js          # Individual todo item with poem generation
├── todo-list.js     # List of all todos
└── layout.js        # Page layout wrapper

styles/
├── todo.module.css      # Styles for todo items
├── todo-list.module.css # Styles for the todo list container
└── layout.module.css    # Styles for the layout
```

---
## Problem 1: Missing CSS Styles

### What Happened?
When clicking "Generate Poem", the poem text appeared **inline** next to the todo item instead of in a popup overlay.

### The Root Cause
The JavaScript component (`todo.js`) referenced CSS class names that **didn't exist** in the CSS file!

**In `todo.js`, we had:**
```jsx
<div className={styles.poemBox}>      {/* This class didn't exist! */}
  <button className={styles.closeButton}>  {/* This class didn't exist! */}
  <div className={styles.poem}>        {/* This class didn't exist! */}
```

**But in `todo.module.css`, we only had:**
```css
.todoInput { ... }
.toDoRow { ... }
.deleteBtn { ... }
/* Missing: .poemBox, .closeButton, .poem, .generatePoemBtn */
```

### Why This Matters
In **CSS Modules** (files ending in `.module.css`), if you reference a class that doesn't exist, the element simply gets **no styling**. It renders with default browser styles, which means:
- No positioning (appears inline in the document flow)
- No background color
- No borders or shadows

### The Fix
We added the missing CSS classes:

```css
.generatePoemBtn {
    padding: 5px 10px;
    background-color: #4a90d9;
    color: white;
    border: none;
    border-radius: 4px;
    cursor: pointer;
}

.poemBox {
    position: fixed;           /* Position relative to viewport */
    top: 50%;                  /* Center vertically */
    left: 50%;                 /* Center horizontally */
    transform: translate(-50%, -50%);  /* Adjust for element's own size */
    background-color: white;
    border: 2px solid #333;
    z-index: 1000;             /* Appear above other content */
}

.closeButton { ... }
.poem { ... }
```

### Key Lesson: Always Verify CSS Class Names Match

When using CSS Modules in React:

| In your JSX | Must exist in CSS file |
|-------------|------------------------|
| `className={styles.poemBox}` | `.poemBox { ... }` |
| `className={styles.closeButton}` | `.closeButton { ... }` |

**Tip:** CSS Modules won't throw an error if a class is missing - your element will just have no styles!

---
## Problem 2: `position: fixed` Not Working

### What Happened?
Even after adding the CSS with `position: fixed`, the popup **still appeared inline** instead of centered on the screen.

### The Root Cause
The popup element was **nested inside a flex container**.

**Original HTML structure:**
```jsx
<div className={styles.toDoRow}>     {/* display: flex */}
  <input type="checkbox" />
  <input type="text" />
  <button>Generate Poem</button>
  <button>Delete</button>
  
  {/* Popup was INSIDE the flex container! */}
  <div className={styles.poemBox}>   
    ...
  </div>
</div>
```

### Why `position: fixed` Can Fail

`position: fixed` is supposed to position an element relative to the **browser viewport**. However, it can break when an ancestor element has:

1. `transform` property (e.g., `transform: translate()`)
2. `perspective` property
3. `filter` property
4. `will-change` property set to `transform`

Additionally, being inside a **flex container** can cause unexpected layout behavior.

### First Attempt: Move Outside the Flex Container
We tried moving the popup outside the flex row using React fragments:

```jsx
return (
  <>                                    {/* React Fragment */}
    <div className={styles.toDoRow}>    {/* Flex container */}
      <input type="checkbox" />
      <input type="text" />
      <button>Generate Poem</button>
      <button>Delete</button>
    </div>
    
    {/* Popup is now OUTSIDE the flex container */}
    {isPoemVisible && (
      <div className={styles.poemBox}>...</div>
    )}
  </>
);
```

**But this still didn't work!** The parent components further up the tree were still affecting the fixed positioning.

---
## Problem 2 Solution: React Portals

### What is a React Portal?

A **Portal** lets you render a component's children into a different part of the DOM tree, **outside** of the parent component hierarchy.

```
Normal rendering:              With Portal:
                              
<body>                        <body>
  <div id="root">               <div id="root">
    <App>                         <App>
      <TodoList>                    <TodoList>
        <Todo>                        <Todo>
          <PoemPopup/>   ──────────────> (empty, portal used)
        </Todo>                       </Todo>
      </TodoList>                   </TodoList>
    </App>                        </App>
  </div>                        </div>
</body>                         <PoemPopup/>  <── Rendered here!
                              </body>
```

### How We Implemented It

**Step 1: Import the required functions**
```jsx
import { useState, useEffect } from 'react'
import { createPortal } from 'react-dom'
```

**Step 2: Track when component is mounted (for SSR compatibility)**
```jsx
const [mounted, setMounted] = useState(false);

useEffect(() => {
  setMounted(true);  // Only true after component mounts in browser
}, []);
```

**Step 3: Use createPortal to render the popup**
```jsx
{isPoemVisible && mounted && createPortal(
  <>                                           {/* Content to render */}
    <div className={styles.overlay}></div>
    <div className={styles.poemBox}>...</div>
  </>,
  document.body                                {/* Where to render it */}
)}
```

### Why Check `mounted`?

Next.js uses **Server-Side Rendering (SSR)**, which means components first render on the server where `document.body` doesn't exist!

```jsx
// This would crash on the server:
createPortal(<Popup/>, document.body)  // document is undefined on server!

// This is safe:
{mounted && createPortal(<Popup/>, document.body)}
// mounted is false on server, so createPortal never runs
```

### The Overlay
We also added a semi-transparent **overlay** behind the popup:

```css
.overlay {
    position: fixed;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    background-color: rgba(0, 0, 0, 0.5);  /* 50% transparent black */
    z-index: 999;                           /* Below popup (1000) */
}
```

This overlay:
- Dims the background content
- Indicates the popup is a modal (focused interaction)
- Can be clicked to close the popup

---
## Problem 3: Delete Icon Overflowing the Container

### What Happened?
After fixing the popup, the delete (trash) icon appeared **outside** the main container border.

### The Root Cause
The container was too **narrow** for all the elements in a todo row.

**Original widths:**
```
Container: 300px

Todo row contents:
├── Checkbox:        ~20px
├── Text input:      194px
├── Generate button: ~100px (with padding)
├── Delete button:   ~30px
└── Margins:         ~20px
────────────────────────────
Total:              ~364px   ← Exceeds 300px!
```

### The Fix
We increased the container width and adjusted all related elements:

| Element | Before | After |
|---------|--------|-------|
| Container | 300px | 500px |
| Main input | 260px | 460px |
| Todo input | 194px | 300px |
| Layout | 300px | 500px |

**Files changed:**
- `todo-list.module.css` - container and main input widths
- `todo.module.css` - todo input width
- `layout.module.css` - layout wrapper width

### Key Lesson: Plan Your Layout Widths

When using flexbox rows, always ensure:

```
Container Width >= Sum of all child widths + margins + padding
```

**Tips for avoiding overflow:**
1. Use browser DevTools to inspect element widths
2. Consider using `flex-wrap: wrap` to allow wrapping
3. Use `overflow: hidden` on container (hides overflow, but may not be desired)
4. Use percentage widths or `flex-grow` for flexible layouts

---
## Summary of All Fixes

| Problem | Cause | Solution |
|---------|-------|----------|
| Poem appeared inline | Missing CSS classes | Added `.poemBox`, `.closeButton`, `.poem`, `.generatePoemBtn` styles |
| `position: fixed` not working | Popup inside flex container with parent CSS interference | Used React Portal to render to `document.body` |
| Delete icon outside container | Container too narrow (300px) for all elements | Widened container to 500px and adjusted child widths |

---
## Final Code: `todo.js`

```jsx
import Image from 'next/image'
import styles from '../styles/todo.module.css'
import { useState, useEffect } from 'react'
import { createPortal } from 'react-dom'

export default function ToDo(props) {
  const { todo, onChange, onDelete } = props;
  const [poem, setPoem] = useState(null);
  const [isPoemVisible, setIsPoemVisible] = useState(false);
  const [mounted, setMounted] = useState(false);

  useEffect(() => {
    setMounted(true);  // For SSR compatibility
  }, []);

  async function generatePoem(id) {
    const res = await fetch(`${process.env.NEXT_PUBLIC_API_URL}/todos/write-poem/${id}`, {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
    });
    if (res.ok) {
      const data = await res.json();
      setPoem(data.poem);
      setIsPoemVisible(true);
    }
  }

  function closePoemBox() {
    setIsPoemVisible(false);
  }

  return (
    <>  {/* Fragment keeps todo row and portal siblings */}
      <div className={styles.toDoRow}>
        {/* ... checkbox, input, buttons ... */}
      </div>
      
      {/* Portal renders popup directly to document.body */}
      {isPoemVisible && mounted && createPortal(
        <>
          <div className={styles.overlay} onClick={closePoemBox}></div>
          <div className={styles.poemBox}>
            <button className={styles.closeButton} onClick={closePoemBox}>
              &times;
            </button>
            <div className={styles.poem}>
              <p>{poem}</p>
            </div>
          </div>
        </>,
        document.body
      )}
    </>
  );
}
```

---
## Key Takeaways for Beginners

1. **Always check that CSS class names match** between your JSX and CSS files

2. **`position: fixed` has limitations** - it can break when ancestor elements have `transform`, `filter`, or `perspective` properties

3. **Use React Portals for modals/popups** - they render outside the component tree, avoiding CSS inheritance issues

4. **Plan container widths carefully** - ensure parent containers are wide enough for all child elements

5. **Use browser DevTools** - inspect elements to see actual computed styles and identify missing CSS